In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import os

# Generate loom files

In [2]:
for seed in range(1, 11):
    base_dir = './Sim/Sim%d' % seed
    count = pd.read_table('%s/counts.txt' % base_dir)
    cellinfo = pd.read_table('%s/cellinfo.txt' % base_dir)
    geneinfo = pd.read_table('%s/geneinfo.txt' % base_dir)
    adata = sc.AnnData(count.values)
    adata.obs = cellinfo[['Batch', 'Group', 'ExpLibSize']].rename(columns={'Batch': 'batch', 'Group': 'celltype'})
    adata.var = geneinfo
    adata.var_names.name = 'Gene'
    adata.obs_names.name = 'CellID'
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)
    adata.write_loom('%s/Sim%d_raw.loom' % (base_dir, seed))

# Generate subsampled files for different scenarios

In [18]:
batch1_prop_cells = [1/2, 1/4, 1/8]
rare1_prop_cells = [1/2, 1/5, 1/10]
common_cells = [5, 3, 1]

In [20]:
for seed in range(1, 11):
    base_dir = './Sim/Sim%d' % seed
    adata = sc.read_loom('%s/Sim%d_raw.loom' % (base_dir, seed))
    for number in common_cells:
        os.mkdir('%s/common_%d' % (base_dir, number))
        common_ct = ['Group%d' % i for i in range(1, number+1)]
        index0 = adata.obs_names[adata.obs.celltype.isin(common_ct)]
        specific_number = 7 - number
        # Get specific cell types for Batch1
        specific_ct = ['Group%d' % i for i in range(number+1, int(number+1+specific_number/2))]
        index1 = adata.obs_names[adata.obs.celltype.isin(specific_ct) & (adata.obs.batch=='Batch1')]
        # Get specific cell types for Batch2
        specific_ct = ['Group%d' % i for i in range(int(number+1+specific_number/2), 7+1)]
        index2 = adata.obs_names[adata.obs.celltype.isin(specific_ct) & (adata.obs.batch=='Batch2')]
        adata_sub = adata[index0.union(index1).union(index2)]
        print(pd.crosstab(adata_sub.obs.celltype, adata_sub.obs.batch))
        adata_sub.write_loom('%s/common_%d/Sim%d_raw.loom' % (base_dir, number, seed))

batch     Batch1  Batch2
celltype                
Group1       243     224
Group2       239     230
Group3       234     219
Group4       218     220
Group5       213     236
Group6       205       0
Group7         0     205
batch     Batch1  Batch2
celltype                
Group1       243     224
Group2       239     230
Group3       234     219
Group4       218       0
Group5       213       0
Group6         0     236
Group7         0     205
batch     Batch1  Batch2
celltype                
Group1       243     224
Group2       239       0
Group3       234       0
Group4       218       0
Group5         0     236
Group6         0     236
Group7         0     205


In [21]:
for seed in range(1,11):
    base_dir = './Sim/Sim%d' % seed
    adata = sc.read_loom('%s/Sim%d_raw.loom' % (base_dir, seed))
    total = np.sum(adata.obs.celltype == 'Group1')
    for number in rare1_prop_cells:
        os.mkdir('%s/rare1_%.1f' % (base_dir, number))
        obs_names = np.random.choice(adata.obs_names[adata.obs.celltype == 'Group1'], int(total*(1-number)), replace=False)
        adata_sub = adata[~adata.obs_names.isin(obs_names)]
        print(adata_sub.obs.celltype.value_counts())
        adata_sub.write_loom('%s/rare1_%.1f/Sim%d_raw.loom' % (base_dir, number, seed))

Group2    469
Group3    453
Group5    449
Group6    441
Group4    438
Group7    414
Group1    234
Name: celltype, dtype: int64
Group2    469
Group3    453
Group5    449
Group6    441
Group4    438
Group7    414
Group1     94
Name: celltype, dtype: int64
Group2    469
Group3    453
Group5    449
Group6    441
Group4    438
Group7    414
Group1     47
Name: celltype, dtype: int64


In [22]:
for seed in range(1,11):
    base_dir = './Sim/Sim%d' % seed
    adata = sc.read_loom('%s/Sim%d_raw.loom' % (base_dir, seed))
    batch1_n = np.sum(adata.obs.batch == 'Batch1')
    for number in batch1_prop_cells:
        os.mkdir('%s/batch1_%d' % (base_dir, number))
        obs_names = np.random.choice(adata.obs_names[adata.obs.batch == 'Batch1'], 
                                     int((1-number)*batch1_n), replace=False)
        adata_sub = adata[~adata.obs_names.isin(obs_names)]
        print(adata_sub.obs.batch.value_counts())
        adata_sub.write_loom('%s/batch1_%.3f/Sim%d_raw.loom' % (base_dir, number, seed))

Batch2    1570
Batch1     781
Name: batch, dtype: int64
Batch2    1570
Batch1     391
Name: batch, dtype: int64
Batch2    1570
Batch1     196
Name: batch, dtype: int64
